In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione da ottimizzare

In [2]:
function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

boundingbox (generic function with 1 method)

## Dati in input

In [3]:
b=[[],[]]
EV=[[1,1]]

for i=1:60
           push!(b[1],(1.0 + i*2.0))
           push!(b[2],(1.0 + i*2.0))
           push!(b[1],(4.0 + i*2.0))
           push!(b[2],(1.0 + i*2.0))
           push!(b[1],(1.0 + i*2.0))
           push!(b[2],(4.0 + i*2.0))
           push!(b[1],(4.0 + i*2.0))
           push!(b[2],(4.0 + i*2.0))
           push!(EV,[1+4*(i-1),2+4*(i-1)])
           push!(EV,[1+4*(i-1),3+4*(i-1)])
           push!(EV,[2+4*(i-1),4+4*(i-1)])
           push!(EV,[3+4*(i-1),4+4*(i-1)])
end

V = permutedims(reshape(hcat(b...), (length(b[1]), length(b))))
filter!(e->e!=[1,1],EV)

240-element Array{Array{Int64,1},1}:
 [1, 2]
 [1, 3]
 [2, 4]
 [3, 4]
 [5, 6]
 [5, 7]
 [6, 8]
 [7, 8]
 [9, 10]
 [9, 11]
 [10, 12]
 [11, 12]
 [13, 14]
 ⋮
 [229, 230]
 [229, 231]
 [230, 232]
 [231, 232]
 [233, 234]
 [233, 235]
 [234, 236]
 [235, 236]
 [237, 238]
 [237, 239]
 [238, 240]
 [239, 240]

## 0) Benchmark vecchia funzione

In [4]:
@btime boundingbox(V)

  109.454 μs (2031 allocations: 46.81 KiB)


([3.0; 3.0], [124.0; 124.0])

## 1) Controllo se la funzione è type unstable

In [5]:
@code_warntype boundingbox(V)

Variables
  #self#::Core.Compiler.Const(boundingbox, false)
  vertices::Array{Float64,2}
  #1::var"#1#3"
  #2::var"#2#4"
  minimum::Any
  maximum::Any

Body::Tuple{Any,Any}
1 ─       (#1 = %new(Main.:(var"#1#3")))
│   %2  = #1::Core.Compiler.Const(var"#1#3"(), false)
│   %3  = (:dims,)::Core.Compiler.Const((:dims,), false)
│   %4  = Core.apply_type(Core.NamedTuple, %3)::Core.Compiler.Const(NamedTuple{(:dims,),T} where T<:Tuple, false)
│   %5  = Core.tuple(2)::Core.Compiler.Const((2,), false)
│   %6  = (%4)(%5)::Core.Compiler.Const((dims = 2,), false)
│   %7  = Core.kwfunc(Main.mapslices)::Core.Compiler.Const(Base.var"#mapslices##kw"(), false)
│         (minimum = (%7)(%6, Main.mapslices, %2, vertices))
│         (#2 = %new(Main.:(var"#2#4")))
│   %10 = #2::Core.Compiler.Const(var"#2#4"(), false)
│   %11 = (:dims,)::Core.Compiler.Const((:dims,), false)
│   %12 = Core.apply_type(Core.NamedTuple, %11)::Core.Compiler.Const(NamedTuple{(:dims,),T} where T<:Tuple, false)
│   %13 = Core.tuple(

La funzione NON è type unstable in quanto ritorna la stringa

    Body::Tuple{Any,Any}

## Parallelizzazione codice e uso dei threads  
  
La funzione originaria utilizzava la funzione 'mapslices', che non è parallelizzabile. Abbiamo quindi riscritto completamente la funzione in modo da renderla accessibile alla macro @threads.

In [6]:
using Base.Threads
function boundingbox2(vertices::Lar.Points)
    d=size(vertices)[1]
    numPoints=size(vertices)[2]
    #inizializzo gli array da ritornare [xMin, yMin, zMin] e [xMax, yMax, zMax]
    mins = zeros(d,1)
    maxs = zeros(d,1)
    for i=1:d
        mins[i]=vertices[i]
        maxs[i]=vertices[i]
    end
    @threads for i=2:numPoints
        @threads for j=1:d
            if(vertices[j+d*(i-1)] > maxs[j])
                maxs[j] = vertices[j+d*(i-1)]
            end
            if(vertices[j+d*(i-1)] < mins[j])
                mins[j] = vertices[j+d*(i-1)]
            end
        end
    end

    return (mins,maxs)
end
print("Numero di threads allocati :")
println(nthreads())
@btime boundingbox2(V)

Numero di threads allocati :2
  19.063 μs (492 allocations: 20.45 KiB)


([3.0; 3.0], [124.0; 124.0])